In [9]:
import torch

In [10]:

if torch.backends.mps.is_available():
    torch_device = torch.device("mps")
    x = torch.ones(1, device=torch_device)
    print (x)
else:
    torch_device = torch.device("cpu")
    print ("MPS device not found.")

tensor([1.], device='mps:0')


## Module

In [11]:
import torch.nn as nn

EMBEDDING_DIM = 3
CONTEXT_LENGTH = 4
QKV_DIM = 2

class CausalSelfAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length=CONTEXT_LENGTH, dropout=0.5, qkv_bias=False):
        super().__init__()
        self.d_in = d_in
        self.d_out = d_out
        self.w_q = nn.Linear(d_in, d_out, bias=qkv_bias).to(torch_device)
        self.w_k = nn.Linear(d_in, d_out, bias=qkv_bias).to(torch_device)
        self.w_v = nn.Linear(d_in, d_out, bias=qkv_bias).to(torch_device)
        self.dropout = nn.Dropout(dropout).to(torch_device)
        self.register_buffer(
            'mask', 
            torch.triu(
                torch.ones(context_length, context_length), 
                diagonal=1,
            ).to(torch_device)
        )

    def forward(self, x):
        num_tokens = x.shape[-2]
        queries = self.w_q(x)
        keys = self.w_k(x)
        attn_scores = queries @ keys.transpose(-2, -1)
        causal_attn_scores = attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        causal_attn_weights = torch.softmax(causal_attn_scores*(self.d_out**0.5), dim=-1)
        causal_attn_weights = self.dropout(causal_attn_weights)
        values = self.w_v(x)
        context = causal_attn_weights @ values
        return context

In [12]:
import torch.nn as nn

EMBEDDING_DIM = 3
CONTEXT_LENGTH = 4
QKV_DIM = 2

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = d_out//num_heads
        self.w_q = nn.Linear(d_in, d_out, bias=qkv_bias).to(torch_device)
        self.w_k = nn.Linear(d_in, d_out, bias=qkv_bias).to(torch_device)
        self.w_v = nn.Linear(d_in, d_out, bias=qkv_bias).to(torch_device)
        self.w_o = nn.Linear(d_out, d_out).to(torch_device)
        self.dropout = nn.Dropout(dropout).to(torch_device)
        self.register_buffer(
            'mask', 
            torch.triu(
                torch.ones(context_length, context_length), 
                diagonal=1,
            ).to(torch_device)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.w_k(x)
        queries = self.w_q(x)
        values = self.w_v(x)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim).transpose(-3, -2)
        # TODO: KV Cache Optimization
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim).transpose(-3, -2)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim).transpose(-3, -2)
        attn_scores = queries @ keys.transpose(-2, -1)
        causal_attn_scores = attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        causal_attn_weights = torch.softmax(causal_attn_scores*(keys.shape[-1]**0.5), dim=-1)
        causal_attn_weights = self.dropout(causal_attn_weights)
        context = (causal_attn_weights @ values).transpose(-3, -2)
        context = context.contiguous().view(b, num_tokens, self.num_heads*self.head_dim)
        context = self.w_o(context)
        return context


## Model

In [13]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False,
}

In [14]:
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.scale = nn.Parameter(torch.ones(d_model))
        self.shift = nn.Parameter(torch.zeros(d_model))

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True, unbiased=False)
        x = (self.scale * (x - mean) / (std + self.eps)) + self.shift
        return x

In [15]:
class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (
            1 + torch.tanh(
                torch.sqrt(torch.tensor(2.0/torch.pi, device=torch_device)) * (x  + 0.044715 * x**3)
            )
        )

In [16]:
class FeedForward (nn.Module):
    def __init__(self, cfg): 
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], cfg["emb_dim"]*4).to(torch_device),
            GELU().to(torch_device),
            nn.Linear(cfg["emb_dim"]*4, cfg["emb_dim"]).to(torch_device),
        )

    def forward(self, x):
        return self.layers(x)

In [17]:
class TransformerBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config["emb_dim"]).to(torch_device)
        self.attn = MultiHeadAttention(config["emb_dim"], config["emb_dim"], config["context_length"], config["drop_rate"], config["n_heads"], config["qkv_bias"]).to(torch_device)
        self.drop = nn.Dropout(config["drop_rate"]).to(torch_device)
        self.ln2 = nn.LayerNorm(config["emb_dim"]).to(torch_device)
        self.ff = FeedForward(config).to(torch_device)

    def forward(self, x):
        x = self.drop(self.attn(self.ln1(x))) + x
        x = self.drop(self.ff(self.ln2(x))) + x
        return x

In [18]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"]).to(torch_device)
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"]).to(torch_device)
        self.drop_emb = nn.Dropout(cfg["drop_rate"]).to(torch_device)
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        ).to(torch_device)
        self.final_norm = LayerNorm(cfg["emb_dim"]).to(torch_device)
        # GPT2 uses tied weights for the embedding and output layers
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False).to(torch_device)

    def forward(self, in_idx):
        in_idx = in_idx.to(torch_device)
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=torch_device))
        x = self.drop_emb(tok_embeds + pos_embeds)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [119]:
def generate_text_simple(model, token_ids, max_new_tokens, context_size):
    logits = None
    for i in range(max_new_tokens):
        print(f"Generating new token #{i+1}")
        context_token_ids = token_ids[:, -context_size:]
        with torch.no_grad():
            logits = model(context_token_ids)

        logits = logits[:, -1, :]
        probas = torch.softmax(logits, dim=-1)
        token_id_next = torch.argmax(probas, dim=1, keepdim=True)  # Pure Greed
        token_ids = torch.cat((token_ids, token_id_next), dim=1) 

    return token_ids

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special=set(['<|endoftext|>']))
    encoded_tensor = torch.tensor(encoded).unsqueeze(0).to(torch_device)
    return encoded_tensor

In [120]:
token_ids = torch.stack([
    text_to_token_ids("The quick brown fox", tokenizer),
    text_to_token_ids("Four score and seven", tokenizer)
], dim=0).squeeze(1)
target_ids = torch.stack([
    text_to_token_ids("The quick brown fox jumps over the lazy dog", tokenizer),
    text_to_token_ids("Four score and seven years ago our fathers brought", tokenizer),
], dim=0).squeeze(1)
num_to_predict = len(target_ids[0]) - len(token_ids[0])
print(num_to_predict)
gpt.eval()
out = generate_text_simple(
    model=gpt,
    token_ids=token_ids,
    max_new_tokens=num_to_predict,
    context_size=GPT_CONFIG_124M["context_length"],
)

5
Generating new token #1
Generating new token #2
Generating new token #3
Generating new token #4
Generating new token #5


In [122]:
logits = gpt(out)
logits.shape

torch.Size([2, 9, 50257])

In [123]:
print(tokenizer.decode(out[0].tolist()))
print(tokenizer.decode(out[1].tolist()))


The quick brown fox severaldeveloped IXDetails647
Four score and seven comparativeisiveatham!), medications


In [124]:
def loss_fn(logits, targets):
    vocab_size = logits.shape[-1]
    loss = nn.CrossEntropyLoss()(logits.view(-1, vocab_size), targets.view(-1))
    return loss

In [126]:
logits.shape, target_ids.shape

(torch.Size([2, 9, 50257]), torch.Size([2, 9]))

In [144]:
probas = torch.softmax(logits, dim=-1)
token_probas = probas.gather(dim=2, index=target_ids.unsqueeze(2)).squeeze(2)
torch.mean(-torch.log(token_probas))

tensor(11.0892, device='mps:0', grad_fn=<MeanBackward0>)

In [146]:
loss = loss_fn(logits, target_ids)
perplexity = torch.exp(loss)
print(loss, perplexity)

tensor(11.0892, device='mps:0', grad_fn=<NllLossBackward0>) tensor(65462.2305, device='mps:0', grad_fn=<ExpBackward0>)


In [ ]:
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader
tokenizer = tiktoken.get_encoding("gpt2")

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)
        token_ids = token_ids
        token_ids = torch.tensor(token_ids).to(torch_device)
        token_sequences = token_ids.unfold(0, max_length, stride)
        self.input_ids = token_sequences[:-1]
        self.target_ids = token_sequences[1:]

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

def create_dataloader_v1(
        txt, batch_size=4, max_length=256, 
        stride=128, shuffle=True, drop_last=True,
        num_workers=0,
    ):
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    return DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=shuffle,
        drop_last=drop_last,
    )

with open("../data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

train_ratio = 0.9
split_idx = len(raw_text) * train_ratio
train_data = raw_text[:split_idx]
test_data = raw_text[split_idx:]

dataloader = create_dataloader_v1(raw_text[50:], batch_size=8, max_length=GPT_CONFIG_124M["context_length"], stride=2, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)